In [1]:
import findspark
findspark.init()

In [94]:
import sys
{sys.prefix}

{'C:\\Users\\User\\anaconda3\\envs\\pipeline'}

In [2]:
!conda install -c conda-forge -y --prefix {sys.prefix} requests


EnvironmentLocationNotFound: Not a conda environment: /Users/mdgome/workspace/study/StudyDataEngineering/playadata/middleproject/{sys.prefix}



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.functions import regexp_extract, lit

In [4]:
from datetime import datetime
import requests
import pandas as pd
import json

In [5]:
obs_post_data = pd.read_csv("./관측소 정보.csv")
obs_post_data.head(5)

,Unnamed: 0,obs_post_id,obs_lat,data_type,obs_post_name,obs_lon,obs_object
0,0,DT_0063,35.024178,조위관측소,가덕도,128.810933,"조위,수온,염분,기온,기압,풍속,풍향"
1,1,DT_0032,37.731944,조위관측소,강화대교,126.522222,"조위,기압"
2,2,DT_0031,34.028333,조위관측소,거문도,127.308889,"조위,수온,염분,기온,기압,풍속,풍향"
3,3,DT_0029,34.801389,조위관측소,거제도,128.699167,"조위,수온,염분,기온,기압,풍속,풍향"
4,4,DT_0058,37.560833,조위관측소,경인항,126.601111,"조위,기온,기압,풍속,풍향"


In [7]:
cluster = '118.34.234.79:7077'
spark = SparkSession \
        .builder \
        .master('spark://'+cluster) \
        .appName('tempera_obs') \
        .getOrCreate()

22/02/21 15:54:34 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

In [8]:
import pprint
pprint.pprint(spark.sparkContext.getConf().getAll())
# spark.stop()

[('spark.app.id', 'app-20220220164646-0000'),
 ('spark.app.startTime', '1645426474549'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '192.168.0.8'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'spark://118.34.234.79:7077'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'tempera_obs'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.sql.warehouse.dir',
  'file:/Users/mdgome/workspace/study/StudyDataEngineering/playadata/middleproject/spark-warehouse'),
 ('spark.driver.port', '49562')]


In [58]:
obs_post_list = obs_post_data['obs_post_id'].values.tolist()
obs_post_list[0:5] # 디버깅 용

['DT_0063', 'DT_0032', 'DT_0031', 'DT_0029', 'DT_0058']

In [64]:
service_key="1LJG14JWnAylzKi5GvmuGg=="
now_datetime=datetime.now().strftime('%Y%m%d')
now_datetime # 디버깅 용

'20220218'

In [46]:
for obs in obs_post_list:
    url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
    response = requests.get(url).json()
    df = spark.createDataFrame(response["result"]["data"])
    df = df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))
    # MySQL Data 적재 하는 부분 생성
    

http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0032&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0031&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0029&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0058&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0026&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0049&Date=20220217&ResultType=json

In [65]:
url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json"

In [90]:
# response = urllib.request.urlopen(url)
json_str = response.read().decode("utf-8")
# str1 = response.json()
json_object = json.loads(json_str)

In [103]:
response = requests.get(url).json()

In [112]:
type(response)

dict

In [136]:
# response["result"]["data"]
response["result"]["meta"]

{'obs_post_id': 'DT_0063',
 'obs_last_req_cnt': '19984/20000',
 'obs_lat': '35.024178',
 'obs_post_name': '가덕도',
 'obs_lon': '128.810933'}

In [137]:
temp_obs_id=response["result"]["meta"]["obs_post_id"]
temp_obs_id

'DT_0063'

In [128]:
df = spark.createDataFrame(response["result"]["data"])

In [108]:
# rdd = spark.sparkContext.parallelize([response])

# df = spark.read.option("multiLine","true").json(rdd)

In [129]:
type(df))

pyspark.sql.dataframe.DataFrame

In [130]:
df.printSchema()

root
 |-- air_temp: string (nullable = true)
 |-- record_time: string (nullable = true)



In [ ]:
# json_schema = ArrayType(
#     StructType([StructField('air_temp', DoubleType(), nullable=False),
#                 StructField('record_time', StringType(), nullable=False),
#                 StructField('obs_post_name', StringType(), nullable=False),
#                 StructField('obs_lat', StringType(), nullable=False),
#                 StructField('obs_lon', StringType(), nullable=False)]))

In [131]:
df.show(5)

+--------+-------------------+
|air_temp|        record_time|
+--------+-------------------+
|    -1.9|2022-02-17 00:00:00|
|    -2.1|2022-02-17 00:01:00|
|    -1.9|2022-02-17 00:02:00|
|    -2.2|2022-02-17 00:03:00|
|    -2.2|2022-02-17 00:04:00|
+--------+-------------------+
only showing top 5 rows



In [140]:
df = df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

In [141]:
df.show(5)

+--------+-------------------+-------+
|air_temp|        record_time|     id|
+--------+-------------------+-------+
|    -1.9|2022-02-17 00:00:00|DT_0063|
|    -2.1|2022-02-17 00:01:00|DT_0063|
|    -1.9|2022-02-17 00:02:00|DT_0063|
|    -2.2|2022-02-17 00:03:00|DT_0063|
|    -2.2|2022-02-17 00:04:00|DT_0063|
+--------+-------------------+-------+
only showing top 5 rows



In [143]:
df.count()

1440

In [9]:
spark.stop()